In [85]:
# Importing
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.preprocessing import PolynomialFeatures
import numpy as np

pd.set_option("display.max_columns", None)


df = pd.read_csv("./energy-training.csv")

In [86]:
df

,Year,Month,Day,Hours,Minutes,Time-since-start,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016,5,10,14,50,17267,60,0,25.200000,47.126667,25.600000,43.900000,26.7000,42.090000,25.500000,44.790000,23.963333,49.900000,23.890000,6.823333,24.890000,44.500000,25.29,48.200000,24.100000,47.525714,18.800,750.150000,78.000000,4.500000,40.000000,14.800,41.358725,41.358725
1,2016,5,4,21,50,16445,60,0,23.390000,33.790000,21.856667,33.500000,23.1000,35.500000,23.000000,33.000000,21.100000,40.663333,8.926667,7.963333,22.412000,33.376000,25.00,37.898571,20.390000,34.463333,10.900,764.283333,57.333333,3.000000,40.000000,2.770,9.024852,9.024852
2,2016,3,16,20,20,9380,140,20,22.600000,37.360000,20.200000,37.693333,23.6000,35.826667,21.790000,33.826667,18.700000,44.200000,5.060000,24.733333,20.230000,25.000000,23.60,33.530000,19.600000,36.200000,5.270,765.200000,62.000000,5.666667,23.333333,-1.500,49.291757,49.291757
3,2016,3,19,2,30,9705,50,0,21.790000,36.760000,19.000000,39.790000,21.7900,36.500000,20.066667,34.966667,18.390000,46.790000,5.123333,66.800000,19.633333,31.133333,22.29,43.090000,19.700000,39.560000,4.700,762.300000,92.000000,3.500000,62.500000,3.500,0.102888,0.102888
4,2016,3,11,5,40,8572,50,0,20.500000,36.200000,16.790000,41.560000,20.3900,37.145000,19.890000,34.000000,17.790000,49.966667,0.233333,70.393333,19.472500,35.475000,21.00,43.000000,18.166667,41.090000,-0.767,763.600000,93.666667,1.333333,33.000000,-1.700,8.882974,8.882974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13895,2016,3,30,8,10,11323,110,0,21.500000,40.500000,19.000000,44.090000,22.5000,38.290000,20.790000,39.590000,20.200000,49.266667,8.445000,44.645000,20.926667,37.000000,23.29,44.163333,20.356667,41.200000,7.550,751.683333,82.333333,4.000000,40.000000,4.650,37.772855,37.772855
13896,2016,4,15,16,30,13677,80,0,22.700000,39.290000,21.790000,38.060000,24.1000,36.790000,22.500000,38.345000,20.790000,44.090000,17.260000,4.196667,22.100000,32.500000,23.16,40.400000,20.890000,42.730000,15.500,746.850000,51.833333,8.833333,30.000000,5.580,27.577899,27.577899
13897,2016,3,10,15,40,8488,60,0,21.066667,36.290000,20.100000,35.400000,20.6000,36.466667,21.600000,33.560000,17.700000,45.260000,10.630000,20.356667,20.666667,29.230000,21.29,35.133333,18.100000,40.663333,9.830,759.333333,54.333333,1.666667,22.000000,0.967,20.786560,20.786560
13898,2016,5,20,20,0,18738,70,0,23.500000,44.326667,22.986250,42.600000,24.5000,40.730000,23.600000,42.700000,23.100000,60.176000,17.733333,4.530000,23.100000,39.536000,23.39,45.590000,22.600000,43.790000,17.300,758.100000,56.000000,4.000000,40.000000,8.500,6.127469,6.127469


In [87]:
def data_transform(df):
    df = df.drop(["rv2", "Year"], axis=1)

    df["Temp_diff_T1_T_out"] = df["T1"] - df["T_out"]
    poly_features = PolynomialFeatures(degree=2, include_bias=False)
    poly_columns = ['T_out', 'RH_out']
    df_poly = pd.DataFrame(poly_features.fit_transform(df[poly_columns]), columns=poly_features.get_feature_names_out(poly_columns))
    df = df.drop(poly_columns, axis=1)
    df = pd.concat([df.reset_index(drop=True), df_poly.reset_index(drop=True)], axis=1)


    return df

In [88]:
df = data_transform(df)
df

,Month,Day,Hours,Minutes,Time-since-start,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,Press_mm_hg,Windspeed,Visibility,Tdewpoint,rv1,Temp_diff_T1_T_out,T_out,RH_out,T_out^2,T_out RH_out,RH_out^2
0,5,10,14,50,17267,60,0,25.200000,47.126667,25.600000,43.900000,26.7000,42.090000,25.500000,44.790000,23.963333,49.900000,23.890000,6.823333,24.890000,44.500000,25.29,48.200000,24.100000,47.525714,750.150000,4.500000,40.000000,14.800,41.358725,6.400000,18.800,78.000000,353.440000,1466.400000,6084.000000
1,5,4,21,50,16445,60,0,23.390000,33.790000,21.856667,33.500000,23.1000,35.500000,23.000000,33.000000,21.100000,40.663333,8.926667,7.963333,22.412000,33.376000,25.00,37.898571,20.390000,34.463333,764.283333,3.000000,40.000000,2.770,9.024852,12.490000,10.900,57.333333,118.810000,624.933333,3287.111111
2,3,16,20,20,9380,140,20,22.600000,37.360000,20.200000,37.693333,23.6000,35.826667,21.790000,33.826667,18.700000,44.200000,5.060000,24.733333,20.230000,25.000000,23.60,33.530000,19.600000,36.200000,765.200000,5.666667,23.333333,-1.500,49.291757,17.330000,5.270,62.000000,27.772900,326.740000,3844.000000
3,3,19,2,30,9705,50,0,21.790000,36.760000,19.000000,39.790000,21.7900,36.500000,20.066667,34.966667,18.390000,46.790000,5.123333,66.800000,19.633333,31.133333,22.29,43.090000,19.700000,39.560000,762.300000,3.500000,62.500000,3.500,0.102888,17.090000,4.700,92.000000,22.090000,432.400000,8464.000000
4,3,11,5,40,8572,50,0,20.500000,36.200000,16.790000,41.560000,20.3900,37.145000,19.890000,34.000000,17.790000,49.966667,0.233333,70.393333,19.472500,35.475000,21.00,43.000000,18.166667,41.090000,763.600000,1.333333,33.000000,-1.700,8.882974,21.267000,-0.767,93.666667,0.588289,-71.842333,8773.444445
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13895,3,30,8,10,11323,110,0,21.500000,40.500000,19.000000,44.090000,22.5000,38.290000,20.790000,39.590000,20.200000,49.266667,8.445000,44.645000,20.926667,37.000000,23.29,44.163333,20.356667,41.200000,751.683333,4.000000,40.000000,4.650,37.772855,13.950000,7.550,82.333333,57.002500,621.616667,6778.777777
13896,4,15,16,30,13677,80,0,22.700000,39.290000,21.790000,38.060000,24.1000,36.790000,22.500000,38.345000,20.790000,44.090000,17.260000,4.196667,22.100000,32.500000,23.16,40.400000,20.890000,42.730000,746.850000,8.833333,30.000000,5.580,27.577899,7.200000,15.500,51.833333,240.250000,803.416667,2686.694444
13897,3,10,15,40,8488,60,0,21.066667,36.290000,20.100000,35.400000,20.6000,36.466667,21.600000,33.560000,17.700000,45.260000,10.630000,20.356667,20.666667,29.230000,21.29,35.133333,18.100000,40.663333,759.333333,1.666667,22.000000,0.967,20.786560,11.236667,9.830,54.333333,96.628900,534.096667,2952.111111
13898,5,20,20,0,18738,70,0,23.500000,44.326667,22.986250,42.600000,24.5000,40.730000,23.600000,42.700000,23.100000,60.176000,17.733333,4.530000,23.100000,39.536000,23.39,45.590000,22.600000,43.790000,758.100000,4.000000,40.000000,8.500,6.127469,6.200000,17.300,56.000000,299.290000,968.800000,3136.000000


In [89]:
X = df.drop("Appliances", axis=1)
y = df[["Appliances"]]


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [90]:
model = ExtraTreesRegressor(n_estimators=100, random_state=42)


model.fit(X_train, y_train.values.ravel())

predictions = model.predict(X_test)


# # Calculate RMSE
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)

r2 = r2_score(y_test, predictions)

print(f"\n\nRMSE: {rmse}")
print(f"\n\nR^2: {r2}")



RMSE: 64.2942251566127


R^2: 0.5408827199158395


In [91]:
holdout = pd.read_csv("./energy-holdout.csv")

In [92]:
holdout = data_transform(holdout)

In [93]:
predictions = model.predict(holdout)
df = pd.DataFrame(predictions)
df.to_csv("ICE-appliances-predictions.csv", index=False, header=["Appliances"])